### Importing Libraries

In [24]:
import dlib
import cv2
import numpy as np
import imageio
from scipy.spatial import Delaunay

### Preprocessing


Function taking path input and returning images and their greyscale counterparts

In [25]:
def getImages(path1, path2):
    img_1 = cv2.imread(path1)
    gray_1 = cv2.cvtColor(src=img_1, code=cv2.COLOR_BGR2GRAY)
    img_2 = cv2.imread(path2)
    img_2 = cv2.resize(img_2, (img_1.shape[1],img_1.shape[0]), interpolation = cv2.INTER_CUBIC)
    gray_2 = cv2.cvtColor(src=img_2, code=cv2.COLOR_BGR2GRAY)
    return img_1, gray_1, img_2, gray_2


### DLib Landmark Detection

Function taking gray images and using dlib library to get landmarkpoints and returning 2 lists for both the images

In [26]:
def DLibLandmarks(gray_1, gray_2):
    #loading the dlib model
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    #detetcting the faces
    faces_1 = detector(gray_1)
    faces_2 = detector(gray_2)
    #adding the 8 points of the boundaries
    landmark_points_1 = []
    landmark_points_1.append([0,gray_1.shape[0]-1])
    landmark_points_1.append([0,0])
    landmark_points_1.append([gray_1.shape[1]/2,0])
    landmark_points_1.append([gray_1.shape[1]/2,gray_1.shape[0]-1])
    landmark_points_1.append([0,gray_1.shape[0]/2])
    landmark_points_1.append([gray_1.shape[1]-1,gray_1.shape[0]/2])
    landmark_points_1.append([gray_1.shape[1]-1,0])
    landmark_points_1.append([gray_1.shape[1]-1,gray_1.shape[0]-1])
    #adding the 68 points of the face
    for face in faces_1:
        landmark_1 = predictor(image=gray_1, box=face)
        for n in range(0, 68):
            x = landmark_1.part(n).x
            y = landmark_1.part(n).y
            landmark_points_1.append([x, y])
    #adding the 8 points of the boundaries
    landmark_points_2 = []
    landmark_points_2.append([0,gray_2.shape[0]-1])
    landmark_points_2.append([0,0])
    landmark_points_2.append([gray_2.shape[1]/2,0])
    landmark_points_2.append([gray_2.shape[1]/2,gray_2.shape[0]-1])
    landmark_points_2.append([0,gray_2.shape[0]/2])
    landmark_points_2.append([gray_2.shape[1]-1,gray_2.shape[0]/2])
    landmark_points_2.append([gray_2.shape[1]-1,0])
    landmark_points_2.append([gray_2.shape[1]-1,gray_2.shape[0]-1])
    #adding the 68 points of the face
    for face in faces_2:
        landmark_2 = predictor(image=gray_2, box=face)
        for n in range(0, 68):
            x = landmark_2.part(n).x
            y = landmark_2.part(n).y
            landmark_points_2.append([x, y])
    return landmark_points_1, landmark_points_2


### Delaunay Triangulation

Function to take landmark points and return 2 list of traingle points in the bothe images

In [27]:
def DelaunayTriangulation(landmark_points_1, landmark_points_2):
    #Delaunay Triangulation
    tri_1 = Delaunay(landmark_points_1)
    pt1 = np.array(landmark_points_1)
    pt2 = np.array(landmark_points_2)
    #getting the triangles corresponding to the indexes based on tri1 indexes
    triangles_1 = pt1[tri_1.simplices]
    triangles_2 = pt2[tri_1.simplices]
    return triangles_1, triangles_2

### Combined PreMorph function

A function to performa all the above steps given the path and return the images along with triangle points

In [28]:
def PreMorph(path1, path2):
    img_1, gray_1, img_2, gray_2 = getImages(path1, path2)
    landmark_points_1, landmark_points_2 = DLibLandmarks(gray_1, gray_2)
    triangles_1, triangles_2 = DelaunayTriangulation(landmark_points_1, landmark_points_2)
    return img_1, img_2, triangles_1, triangles_2

## Affine Transformation

A function to get the points on the morph image based on alpha

In [29]:
def morph_points(p1, p2, alpha):
    return (1 - alpha) * p1 + alpha * p2

A function to warp a single image into the structure of the second image given the triangulation

In [30]:
def warp_images(img1, img2, triangles1, triangles2):
    for i in range(0,len(triangles1)):
        # Find bounding rectangle for each triangle
        r1 = cv2.boundingRect(np.float32(triangles1[i]))
        r2 = cv2.boundingRect(np.float32(triangles2[i]))
        # Offset points by left top corner of the respective rectangles
        t1Crop = triangles1[i] - [r1[0], r1[1]]
        t2Crop = triangles2[i] - [r2[0], r2[1]]
        warp_matrix = cv2.getAffineTransform(np.float32(t1Crop), np.float32(t2Crop))
        # Apply warpImage to small rectangular patches
        img1_crop = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
        img2_crop = cv2.warpAffine(img1_crop, warp_matrix, (r2[2], r2[3]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
        # Get mask by filling triangle
        mask = np.zeros((r2[3], r2[2], 3), dtype = np.float32)
        cv2.fillConvexPoly(mask, np.int32(t2Crop), (1.0, 1.0, 1.0), 16, 0)
        img2Rect = np.copy(img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]])
        img2Rect = img2Rect * (1 - mask) + img2_crop * mask
        # Copy triangular region of the rectangular patch to the output image
        img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2Rect
        
    return img2  

A Function to Morph images given their triangulation and alpha

In [31]:
def morph_images(img_1, img_2, triangles_1, triangles_2,alpha):
    #Finds the morph points 
    morph_points_ = morph_points(triangles_1,triangles_2,alpha)
    #creates a copy of images
    t1 = img_1.copy()
    t2 = img_2.copy()   
    #warps first on the morph image
    morphed_image_1 = warp_images(img_1, t2, triangles_1, morph_points_)
    #warps second on the moprh image
    morphed_image_2 = warp_images(img_2, t1, triangles_2,  morph_points_)
    #takes weighted average of the pixels
    morphed_image = morphed_image_1 * (1 - alpha) + morphed_image_2 * alpha
    #preprocessing for gif generation
    morphed_image = np.uint8(morphed_image)
    morphed_image_rgb = cv2.cvtColor(morphed_image, cv2.COLOR_BGR2RGB)
    return morphed_image_rgb

### File Read function

Function to read File input and return the triangulation

In [34]:
def FileRead():
    file = open("landmarks.txt", "r")
    lines = file.readlines()
    points_1 = []
    points_2 = []
    for line in lines:
        line = line.strip()
        line = [float(x) for x in line.split(" ")]
        points_1.append([line[0], line[1]])
        points_2.append([line[2], line[3]])

    file_triangles_1, file_triangles_2 = DelaunayTriangulation(points_1, points_2)
    file.close()
    return file_triangles_1, file_triangles_2


## Final Code Block

Final Code block that has global variables and needs to run

This is concise since any outside changes won't affect any function calls

uncomment the second line if triangulation is to be done from file Input

In [35]:
img_1, img_2, triangles_1, triangles_2 = PreMorph('face1.jpg','face2.jpg')
# triangles_1, triangles_2 = FileRead() 
morphed_images = []
for i in range(0,61):
    morphed_images.append(morph_images(img_1,img_2,triangles_1,triangles_2,i/60))
imageio.mimwrite('result.gif',morphed_images,fps=10)
